In [ ]:
from qiskit import QuantumRegister,QuantumCircuit,transpile
import numpy as np
import matplotlib.pyplot as plt
from QLBM import duplicate_density_field

In [ ]:
field_encoding_qubit_number = np.arange(1,12)
gate_counts_list_baseline_initialization = []
gate_counts_list_improved_initialization = []

In [ ]:
# Loop over different numbers of qubits for field encoding
for qubit_number in field_encoding_qubit_number:
    # Define grid dimensions based on qubit number
    N_POINTS_X = 2**qubit_number
    N_POINTS_Y = 2**1
    N_POINTS_Z = 2**1

    # Initialize the field and a zero field
    Psi_init = np.full((N_POINTS_X, N_POINTS_Y, N_POINTS_Z), 0.1)
    zeros_ = np.zeros((N_POINTS_X, N_POINTS_Y, N_POINTS_Z))

    # Create and normalize baseline and improved state vectors
    concatenated_baseline = np.concatenate([Psi_init] * 27 + [zeros_] * 5, axis=None)
    Normalized_psi_baseline = concatenated_baseline / np.linalg.norm(concatenated_baseline)
    Normalized_psi_improved = Psi_init.flatten() / np.linalg.norm(Psi_init.flatten())

    # Quantum registers for x, y, z axes and field velocities (specific to D3Q27 model)
    qx = QuantumRegister(qubit_number, 'qx')
    qy = QuantumRegister(1, 'qy')
    qz = QuantumRegister(1, 'qz')
    qf = QuantumRegister(5, 'qf')  # for D3Q27 velocities

    # Initialize quantum circuits for baseline and improved models
    qc_baseline = QuantumCircuit(qx, qy, qz, qf)
    qc_improved = QuantumCircuit(qx, qy, qz, qf)

    # Apply initializations
    qc_baseline.initialize(Normalized_psi_baseline, qc_baseline.qubits)
    qc_improved.initialize(Normalized_psi_improved, [qx, qy, qz])
    duplicate_density_field(qc=qc_improved, num_velocities=27)

    # Transpile circuits with optimization level 1
    result_baseline = transpile(qc_baseline, basis_gates=['id', 'rz', 'sx', 'cx', 'x'], optimization_level=1)
    result_improved = transpile(qc_improved, basis_gates=['id', 'rz', 'sx', 'cx', 'x'], optimization_level=1)

    # Collect gate counts for each approach
    gate_counts_baseline = result_baseline.count_ops()
    gate_counts_improved = result_improved.count_ops()
    gate_counts_list_baseline_initialization.append(gate_counts_baseline)
    gate_counts_list_improved_initialization.append(gate_counts_improved)

In [ ]:
# Convert the list of gate count dictionaries to NumPy arrays
gate_arrays_improved = [np.array(list(gate_counts.values())) for gate_counts in gate_counts_list_improved_initialization]
gate_arrays_basline = [np.array(list(gate_counts_2.values())) for gate_counts_2 in gate_counts_list_baseline_initialization]


In [ ]:
plt.rcParams.update({'font.size': 22})
plt.figure(2,figsize=(10, 6), dpi=300)
for i in range(len(gate_arrays_improved[0])-2):
    plt.plot(field_encoding_qubit_number+4, [gate_array[i] for gate_array in gate_arrays_improved],'b--' if i == 0 else 'g--' if i == 1 else 'k--')

for i in range(len(gate_arrays_basline[0])):
    plt.plot(field_encoding_qubit_number+4, [gate_array_2[i] for gate_array_2 in gate_arrays_basline], 'b' if i == 0 else 'g' if i == 1 else 'k')

plt.grid()
plt.legend(['RZ optimized','SX optimized','CX optimized','RZ baseline', 'SX baseline', 'CX baseline'],fontsize=16)
plt.ylabel('Gate number',fontsize=18)
plt.xlabel('Qubit number',fontsize=18)
plt.yscale('log')
plt.setp(plt.gca().lines, linewidth=1.5)
plt.gcf().set_dpi(300)
plt.xlim([5,12])
plt.ylim([1,10**5])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.draw()